# Classificazione di Spambase attraverso l'utilizzo di Random Forest

Fasoli Marco, 5611520

## Teoria: Random Forest

Tra gli algoritmi di machine learning, il modello Random Forest rappresenta una delle soluzioni più affidabili per problemi di classificazione (ma anche di regressione). Si tratta di un metodo di apprendimento ensemble, che combina più laberi decisionali per ottenere un modello complessivo più stabile rispetto ai singoli alberi.

Al posto di far affidamento ad un solo albero decisionale (che solitamente è soggetto all'overfitting), un Random Forest costruisce una foresta di alberi. Ciascun di questi alberi viene addestrato su un sottoinsieme casuale del dataset di training, selezionato mediante la tecnica del bootstrap (campionamento con ripetizione, alcune istanze possono essere visionate più volte mentre altre mai). Ogni nodo dell'albero considera solo un sottoinsieme casuale delle caratteristiche (colonne nel nostro caso) disponibili per decidere la partizione ottimale. Questa casualità introduce varietà tra gli alberi e riduce la correlazione tra di essi, migliorando la generalizzazione.

L'accuratezza del modello viene calcolata confrontando le predizioni del modello con le etichette reali su un set di dati di test, ovvero ogni albero della foresta produce una predizione indipendente e la predizione finale per ogni istanza è ottenuta tramite voto di maggioranza (nel caso della classificazione).

## Pratica

A questo punto possiamo utilizzare questo modello di regressione al fine di classificare le mail e dire se sono spam oppure no. Per far ciò utilizzeremo il dataset offerto da UCI "Spambase".

### Imports

In [7]:
import pandas as pd
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

### Esplorazione e pulizia dataset

Dal momento che UCI spesso offre il dataset utilizzando il formato *.data*, dobbiamo "convertirlo" in csv nel seguente modo: 

In [10]:
# Define the column names based on spambase.names
column_names = [
    'word_freq_make', 'word_freq_address', 'word_freq_all', 'word_freq_3d', 'word_freq_our',
    'word_freq_over', 'word_freq_remove', 'word_freq_internet', 'word_freq_order', 'word_freq_mail',
    'word_freq_receive', 'word_freq_will', 'word_freq_people', 'word_freq_report', 'word_freq_addresses',
    'word_freq_free', 'word_freq_business', 'word_freq_email', 'word_freq_you', 'word_freq_credit',
    'word_freq_your', 'word_freq_font', 'word_freq_000', 'word_freq_money', 'word_freq_hp',
    'word_freq_hpl', 'word_freq_george', 'word_freq_650', 'word_freq_lab', 'word_freq_labs',
    'word_freq_telnet', 'word_freq_857', 'word_freq_data', 'word_freq_415', 'word_freq_85',
    'word_freq_technology', 'word_freq_1999', 'word_freq_parts', 'word_freq_pm', 'word_freq_direct',
    'word_freq_cs', 'word_freq_meeting', 'word_freq_original', 'word_freq_project', 'word_freq_re',
    'word_freq_edu', 'word_freq_table', 'word_freq_conference', 'char_freq_;', 'char_freq_(',
    'char_freq_[', 'char_freq_!', 'char_freq_$', 'char_freq_#',
    'capital_run_length_average', 'capital_run_length_longest', 'capital_run_length_total',
    'spam'  # 1 for spam, 0 for non-spam
]
df = pd.read_csv('spambase.data', header=None, names=column_names)

A questo punto visualizziamo le informazioni di questo dataset e mostriamo alcuni esempi di istanze del dataset.

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4601 entries, 0 to 4600
Data columns (total 58 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   word_freq_make              4601 non-null   float64
 1   word_freq_address           4601 non-null   float64
 2   word_freq_all               4601 non-null   float64
 3   word_freq_3d                4601 non-null   float64
 4   word_freq_our               4601 non-null   float64
 5   word_freq_over              4601 non-null   float64
 6   word_freq_remove            4601 non-null   float64
 7   word_freq_internet          4601 non-null   float64
 8   word_freq_order             4601 non-null   float64
 9   word_freq_mail              4601 non-null   float64
 10  word_freq_receive           4601 non-null   float64
 11  word_freq_will              4601 non-null   float64
 12  word_freq_people            4601 non-null   float64
 13  word_freq_report            4601 

In [13]:
df.sample(n = 5)

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
3689,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.000,0.000,0.0,0.000,0.00,0.000,2.000,4,6,0
4125,0.0,0.0,2.04,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.226,0.000,0.0,0.000,0.00,0.000,1.000,1,8,0
392,0.0,0.0,0.59,0.0,0.59,0.0,0.0,0.59,0.0,0.0,...,0.000,0.105,0.0,0.105,0.42,0.000,3.428,12,72,1
3157,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.000,0.536,0.0,0.000,0.00,0.268,2.529,11,43,0
22,0.0,0.0,0.00,0.0,2.94,0.0,0.0,0.00,0.0,0.0,...,0.404,0.404,0.0,0.809,0.00,0.000,4.857,12,34,1


Adesso cerchiamo se esistono istanze con valori nulli o valori non nel dominio (i.e. numeri negativi).

In [15]:
duplicated_rows = df.duplicated()
num_duplicate_rows = sum(duplicated_rows)
print(f"Number of duplicate rows: {num_duplicate_rows}")

Number of duplicate rows: 391


In [16]:
numerical_cols = df.select_dtypes(include='number').columns
(df[numerical_cols] < 0).sum()

word_freq_make                0
word_freq_address             0
word_freq_all                 0
word_freq_3d                  0
word_freq_our                 0
word_freq_over                0
word_freq_remove              0
word_freq_internet            0
word_freq_order               0
word_freq_mail                0
word_freq_receive             0
word_freq_will                0
word_freq_people              0
word_freq_report              0
word_freq_addresses           0
word_freq_free                0
word_freq_business            0
word_freq_email               0
word_freq_you                 0
word_freq_credit              0
word_freq_your                0
word_freq_font                0
word_freq_000                 0
word_freq_money               0
word_freq_hp                  0
word_freq_hpl                 0
word_freq_george              0
word_freq_650                 0
word_freq_lab                 0
word_freq_labs                0
word_freq_telnet              0
word_fre

Dal momento che vi sono istanze duplicate, procediamo ad eliminare queste righe identiche.

In [18]:
df = df.drop_duplicates()

Leggendo la documentazione di questo dataset, possiamo osservare come vi sono due caratteristiche che garantiscono che una mail non sia spam. Queste due caratteristiche sono: valore di word_freq_george > 0 e valore di word_freq_650 > 0, pertanto procediamo a impostare spam a 0 per queste mail.

In [20]:
df.loc[(df['word_freq_george'] > 0) | (df['word_freq_650'] > 0), 'spam'] = 0

## Modello predittivo

A questo punto possiamo definire il nostro modello che categorizzerà le mail di questo dataset. Procediamo a creare i dataset per il tranining e per il test.

In [23]:
X = df.drop('spam', axis=1)
y = df['spam']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [25]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Adesso passiamo alla fase di testing, ottenendo percentuali sulla accuratezza di questo modello.

In [27]:
y_pred_rf = model.predict(X_test)

In [28]:
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))

Accuracy: 0.9422011084718923

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.96      0.95       750
           1       0.94      0.92      0.93       513

    accuracy                           0.94      1263
   macro avg       0.94      0.94      0.94      1263
weighted avg       0.94      0.94      0.94      1263



Osserviamo che otteniamo una precisione del 94% della classificazione delle mail. Inoltre, utilizziamo il classification report per un analisi più dettagliata. Questa tabella è formata da:
- Precision: di tutte le email che il modello che ha classificato come appartenenti ad una determinata classe, quante lo erano veramente?
- Recall: di tutte le email che erano veramente di questa classe, quante il modello è riuscito ad identificare correttamente?
- F1-score: è la media armonica della precisione e della recall.
- Support: è il numero di email di quella classe nel set di dati su cui è stato calcolato il report.

Invece le righe:
- Accuracy: è la percentuale totale di email classificate correttamente su tutte le email nel set di test.
- Macro Avg: è la media semplice (non pesata) della precisione, recall e F1-score per tutte le classi.
- Weighted Avg: è la media della precisione, recall e F1-score per tutte le classi, pesata in base al support di ogni classe.

In generale possiamo osservare un'alta precisione di classificazione delle email di questo dataset. Come ultimo passo osserviamo alcune delle feature utilizzate dagli alberi che costituiscono la foresta utilizzata.

In [30]:
print("\nFeature Importances:")
print(pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False))


Feature Importances:
char_freq_!                   0.125875
char_freq_$                   0.091839
word_freq_remove              0.089766
capital_run_length_average    0.065329
word_freq_free                0.064630
capital_run_length_longest    0.051251
word_freq_your                0.051214
capital_run_length_total      0.046254
word_freq_hp                  0.036647
word_freq_our                 0.030774
word_freq_george              0.027579
word_freq_000                 0.026431
word_freq_you                 0.025914
word_freq_internet            0.024171
word_freq_money               0.023177
word_freq_edu                 0.019256
word_freq_1999                0.014434
word_freq_all                 0.014423
word_freq_hpl                 0.013448
word_freq_business            0.012335
char_freq_(                   0.011275
word_freq_will                0.010696
word_freq_over                0.009920
word_freq_email               0.008701
word_freq_receive             0.008488
wor

Possiamo osservare che le colonne più importanti per il modello sono quelle relative alle occorrenze dei simboli "!" e "$", come si poteva predire.

### Confronto con Regressione Logistica

In questa sezione confronteremo il Random Forest model con un modello di Regressione Logistica. Utilizzeremo lo stesso split del dataset per un confronto più significativo.

In [34]:
model_lr = LogisticRegression(solver='liblinear', random_state=42)
model_lr.fit(X_train, y_train)

LogisticRegression(random_state=42, solver='liblinear')

In [35]:
y_pred_lr = model_lr.predict(X_test)

A questo punto confrontiamo l'accuratezza di questo modello:

In [37]:
print("--- Logistic Regression Results ---")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lr))

--- Logistic Regression Results ---
Accuracy: 0.9239904988123515

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.95      0.94       750
           1       0.92      0.89      0.90       513

    accuracy                           0.92      1263
   macro avg       0.92      0.92      0.92      1263
weighted avg       0.92      0.92      0.92      1263



Infine confrontiamo i risultati dei due modelli:

In [39]:
print("\n--- Random Forest Results (from previous run) ---")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))


--- Random Forest Results (from previous run) ---
Accuracy: 0.9422011084718923

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.96      0.95       750
           1       0.94      0.92      0.93       513

    accuracy                           0.94      1263
   macro avg       0.94      0.94      0.94      1263
weighted avg       0.94      0.94      0.94      1263



Osservando che il Random Forest model è più preciso nella classificazione delle mail.

## Conclusioni

Il modello Random Forest tende ad essere più preciso rispetto alla Regressione Logistica in quanto:
- Non-linearità vs Linearità: la regressione logistica è un modello lineare, pertanto modella la relazione tra le feature e la probabilità che un'istanza appartenga a una classe utilizzando una combinazione lineare pesata delle feature. Funziona in modo ottimo se le classi sono separabili in modo lineare nello spazio delle feature. Invece il Random Forest è un modello non-lineare, pertanto può creare superfici di decisione molto più complesse e frastagliate, in quanto ciascun albero decide basandosi su una serie di split sulle feature, che non sono vincolati a una relazione lineare.
- Cattura di Iterazioni e Relazioni Complesse: la Regressione Logistica fatica a catturare autnomaticamente le interazioni complesse tra le feature o le relazioni non lineari tra una singola feature e la variabile target. Invece, il Random Forest cattura naturalemente interazioni e relazioni non lineari. Infatti, l'ensemble di molti alberi permette di combinare interazioni locali per creare un modello predittivo robusto e capace di mappare relazioni complesse.